# Choose your own adventure story

### First define the different stories available
- Sherlock
- Alice in Wonderland
- Ulysses


In [1]:
print('hi')

hi


In [2]:
from textblob import TextBlob
from collections import Counter
import nltk
from functools import reduce
import operator
import numpy as np

In [3]:
print('Welcome to a choose yur own adventure story! Below you will pick')

Welcome to a choose yur own adventure story! Below you will pick


In [32]:
story_names = ['Sherlock','Alice in Wonderland','Ulysses']
while True:
    story = input("Which story would you like to inhabit? You can choose Sherlock, Alice in Wonderland, or Ulysses.")
    if story in story_names:
        break

Which story would you like to inhabit? You can choose Sherlock, Alice in Wonderland, or Ulysses.Sherlock


In [33]:
## Open the appropriate story based upon the user input
if story =='Alice in Wonderland':
    with open("../corpora/alice_in_wonderland.txt") as f:
        text = f.read()
elif story =='Sherlock':
    with open("../corpora/sherlock_all.txt") as f:
        text = f.read()
elif story =='Ulysses':
    with open("../corpora/ulysses.txt") as f:
        text = f.read()

In [34]:
#Clean the text

In [35]:
#Token the words, count the frequency of each token, and create a unigram, bigram, and trigram dictionary
blob_text = TextBlob(text)
text_tokens = blob_text.tokens
text_counter_unigram = Counter(text_tokens)
text_counter_bigram = Counter(nltk.bigrams(text_tokens))
text_counter_trigram = Counter(nltk.trigrams(text_tokens))

In [36]:
def prob_distribution(counter):
    """Return the probability based upon a counter dictionary"""
    #increase by 100000 to use numpy random choice
    return {k:value/sum(counter.values()) for k,value in counter.items()}
    
    


# def product(iterable):
#     "Multiply the numbers together.  (Like `sum`, but with multiplication.)"
#     return reduce(operator.mul, iterable, 1)

# def prob_words(words):
#     "Probability of words, assuming each word is independent of others."
#     return product(prob_distrubtion(w) for w in words)

In [37]:
#Probability distributions for different text combinations
#This takes ~10 minutes to run
prob_dist_unigram = prob_distribution(text_counter_unigram)
prob_dist_bigram = prob_distribution(text_counter_bigram)

In [50]:
prob_dist_trigram = prob_distribution(text_counter_trigram)

In [52]:
def cond_prob_word(end_word, first_word):
    "Conditional probability of word, given previous word."
    bigram = (first_word , end_word)
    if prob_dist_bigram[bigram] > 0 and prob_dist_unigram[first_word] > 0:
        
        return prob_dist_bigram[bigram] / prob_dist_unigram[first_word]
    else: # Average the back-off value and zero.
        return prob_dist_unigram[end_word] / 2


In [57]:
def cond_prob_trigram(start,middle,end):
    """Find the conditional probability of a trigram model given the two previous words."""
    trigram = (start ,middle , end)
    bigram = ( start,middle)
    if prob_dist_trigram[trigram] > 0 and prob_dist_bigram[bigram] > 0 and prob_dist_unigram[start] >0:
        
        return prob_dist_trigram[trigram] / prob_dist_bigram[bigram] ##return prob of trigram over first two words
    elif prob_dist_bigram[bigram] > 0 and prob_dist_unigram[start] >0: # Back off to bigram model
        return prob_dist_bigram[bigram] / prob_dist_unigram[start]
    else: #back off to unigram model (three words)
        return prob_dist_unigram[end]
        

In [59]:
prob_dist_trigram[('I', 'saw', 'what')]

7.960262370247724e-06

In [58]:
cond_prob_trigram('I', 'saw', 'what')

0.023255999075869078

In [87]:
def sentence_generator(probability_dict_unigram, probability_dict_bigram,probability_dict_trigram, number_of_sentences):
    """Generate random sentences based upon the probabilities given the the probability_dict"""
    number=0
    generated_text=[]
    character_counter = 0
        
    #Find starting words of sentences
    starting_chars = [item[1]  for item in list(probability_dict_bigram.keys()) if item[0]=="."]
    starting_chars_counter = Counter(starting_chars)
    #print(starting_chars)
    
    #find list of unigram probabilities for starting characters
    starting_prob = prob_distribution(starting_chars_counter)
    
    #Pick an initial starting character
    start_char_index = np.random.choice([i for i in range(len(starting_chars))],1,p=list(starting_prob.values()))
    generated_text.append(starting_chars[start_char_index])
    print(generated_text,'generated text')
    
    while number !=number_of_sentences: #make sure we have this number of sentences
      
        print(generated_text,'generated text')
        
        if len(generated_text)<3:
        
            words_list = list(probability_dict_bigram.keys())
            prev_character=generated_text[character_counter]

            current_word_options = [i[1] for i in text_counter_bigram if i[0]==prev_character] #Find bigrams with prev char
            print(current_word_options, ' current word options')

            prob_bigram_list = []
            for curr_word in current_word_options:
                prob_bigram_list.append(cond_prob_word(curr_word,prev_character))

            #print(prob_bigram_list, ' prob bigram')
            print(sum(prob_bigram_list))

            # weighted choice algorithm
            # http://stackoverflow.com/questions/22722079/choosing-elements-from-python-list-based-on-probability
            # 1) pick random number between 0 and 1
            # 2) walk through the list, subtracting each item from your number as your go
            # 3 ) when you go to 0 or below, pick the current item
            weight = np.random.random()
            print(weight,'weight')
            bigram_word_index = 0

            for index,prob in enumerate(prob_bigram_list):
                weight -=prob
                if weight <0:
                    bigram_word_index=index

            word = current_word_options[bigram_word_index]
        
            generated_text.append(word)
            character_counter+=1 ## go to the next character
            
        elif len(generated_text)>2: ###trigram 
            
            words_list = list(probability_dict_trigram.keys()) ## list of all trigram
            first_character=generated_text[character_counter] # find the previous word (one index away)
            second_character=generated_text[character_counter-1] #find the previous word
            
            current_triword_options= []
            prob_trigram_list = [] #list of conditional probabilities
            for i in text_counter_trigram:
                if i[0]==first_character and i[1]==second_character:
                    current_triword_options = i[2] #the current word to predict
                    print(curr_word , 'current word')
                    prob_trigram_list.append(cond_prob_trigram(curr_word[0],curr_word[1],curr_word[2]))
                    
            # tuple ('The', 'boy', 'went) i[0]=The, i[1]=boy, i[2]=went (we want to predict went)
            print(current_triword_options, ' current word options')



            #print(prob_bigram_list, ' prob bigram')
            print(sum(prob_trigram_list))

            # weighted choice algorithm
            # http://stackoverflow.com/questions/22722079/choosing-elements-from-python-list-based-on-probability
            # 1) pick random number between 0 and 1
            # 2) walk through the list, subtracting each item from your number as your go
            # 3 ) when you go to 0 or below, pick the current item
            weight = np.random.random()
            print(weight,'weight')
            trigram_word_index = 0

            for index,prob in enumerate(prob_trigram_list):
                weight -=prob
                if weight <0:
                    try:
                        word = current_triword_options[trigram_word_index]
                    except:
                        new_weight = np.random.random()
                        
                        
                    trigram_word_index=index
                    break
            try: ##might not find a trigram
                word = current_triword_options[trigram_word_index]
            except: ##try a bigram
                
                word = current_word_options[trigram_word_index]
                
        
            generated_text.append(word)
            
            character_counter+=1 ## go to the next character (skip characters due to trigram model)
            
        
        
        if word ==".": ##end of the sentence
            number+=1
        
        if character_counter==15:
            break

       
    return generated_text

In [170]:
text_counter_bigram[('boy','the')]

0

# TESTING

In [ ]:
cond_prob_word('the','man')

In [88]:
sentence_generator(prob_dist_unigram,prob_dist_bigram,prob_dist_trigram,1)

/Users/jonathanhilgart/anaconda/envs/nlp/lib/python3.5/site-packages/ipykernel/__main__.py:17: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future


['With'] generated text
['With'] generated text
['that', 'an', 'his', 'him', 'the', 'a', 'trembling', 'this', 'these', 'much', 'making', 'my', 'your', 'hardly']  current word options
1.0000079601990053
0.14149390848449417 weight
['With', 'hardly'] generated text
['tell', 'shut', 'have', 'out', 'imagine', 'served', 'spoken', 'flashed', 'yet', 'explain', 'reached', 'visible', 'consider', 'wander', 'pass', 'avoid', 'be', 'expect', 'noticed', 'said', 'take', 'looked', 'from', 'spoke', 'found', 'resist', 'see', 'believe', 'get', 'knows', 'a', 'safe', 'think', 'listened', 'knowing', 'finished']  current word options
1.000007960199005
0.16593479626485896 weight
['With', 'hardly', 'finished'] generated text
[]  current word options
0
0.5112980623644757 weight
['With', 'hardly', 'finished', 'tell'] generated text
[]  current word options
0
0.6633964374523068 weight
['With', 'hardly', 'finished', 'tell', 'tell'] generated text
[]  current word options
0
0.026140209751155385 weight
['With', 'hard

['With',
 'hardly',
 'finished',
 'tell',
 'tell',
 'tell',
 'tell',
 'tell',
 'tell',
 'tell',
 'tell',
 'tell',
 'tell',
 'tell',
 'tell',
 'tell']

In [37]:
l = [1,2,3,4,3,2,334234,2,2,3,3,4,4,3,2,2,2,2]

In [38]:
coun_l = Counter(l)

In [43]:
prob_distribution(coun_l)

{1: 0.05555555555555555,
 2: 0.4444444444444444,
 3: 0.2777777777777778,
 4: 0.16666666666666666,
 334234: 0.05555555555555555}

In [181]:
np.random.random()

0.545593768033727